In [16]:
import hive
import glob
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
def get_destination_from_movestring(move_string, board):
    if move_string[0] in ['\\', '/' ,'-']:
            #left
            pos = move_string[:1]
            if pos == '\\':
                direction = hive.Flat_Directions.NE
            elif pos == '-':
                direction = hive.Flat_Directions.N
            elif pos == '/':
                direction = hive.Flat_Directions.NW
            piece = move_string[1:]
            return board.go_direction(board.get_current_position(piece), direction)
    elif move_string[-1] in ['\\', '/' ,'-']:
        #right
        pos = move_string[-1:]
        if pos == '\\':
            direction = hive.Flat_Directions.SW
        elif pos == '-':
            direction = hive.Flat_Directions.S
        elif pos == '/':
            direction = hive.Flat_Directions.SE
        piece = move_string[:-1]
        return board.go_direction(board.get_current_position(piece), direction)
    else:
        # no movestring ergo beetle
        return board.get_current_position(move_string)

In [3]:
def get_tile_encoding(tile):
    """Quick and dirty method to translate pieces into a simple integer to hopefully
    represent state enough for the IL to understand"""
    total = 0
    if tile.color == hive.Color.Black:
        total += 100
    if tile.insect == hive.Insect.Ant:
        total += 10
    elif tile.insect == hive.Insect.Beetle:
        total += 20
    elif tile.insect == hive.Insect.Grasshopper:
        total += 30
    elif tile.insect == hive.Insect.Spider:
        total += 40
    elif tile.insect == hive.Insect.Queen:
        total += 50
    elif tile.insect == hive.Insect.Ladybug:
        total += 60
    elif tile.insect == hive.Insect.Pillbug:
        total += 70
    elif tile.insect == hive.Insect.Mosquito:
        total += 80
    if tile.number == 1:
        total += 1
    elif tile.number == 2:
        total += 2
    elif tile.number == 3:
        total += 3
    return total

t = hive.Tile.from_string('wG3')
get_tile_encoding(t)

33

In [4]:
def parse_pgn_file(file):
    output = []
    with open(file, 'r+') as f:
        for _ in range(0, 9):
            next(f)
        for line in f:
            line=line.split(' ')[1:]
            line[-1] = line[-1].strip('\n')
            output.append(line)
    return output

In [5]:
from copy import deepcopy

def moves_as_board_states(moves):
    output = []
    board = hive.HiveBoard()
    board.place(hive.Tile.from_string(moves[0][0]), (0,0))
    output.append(deepcopy(list(board.get_pieces())))
    for move in moves[1:]:
        if move[0] != 'pass':
            orig = board.get_current_position(move[0])
            if orig == None:
                board.place(hive.Tile.from_string(move[0]), get_destination_from_movestring(move[1], board))
                output.append(deepcopy(list(board.get_pieces())))
            else:
                board.move(orig, get_destination_from_movestring(move[1], board))
                output.append(deepcopy(list(board.get_pieces())))
        else:
            output.append(deepcopy(list(board.get_pieces())))
    return output, board

In [6]:
# Load PGNs from 2020 to 2023
# will add more though this is enough for testing purposes
all_games = []
for i in glob.glob(r'valid_pgns/202[0-3]_pgn/*.pgn'):
    # do not open files that are empty or too small to have valid games
    if os.stat(i).st_size > 200:
        print(i)
        all_games.append(parse_pgn_file(i))

valid_pgns/2023_pgn/T!HV-smburgan-Kaur50-2023-01-08-2039.pgn
valid_pgns/2023_pgn/U!HV-ringersoll-playertwo-2023-02-28-1249.pgn
valid_pgns/2023_pgn/T!HV-grajjio-Marko60-2023-05-09-1944.pgn
valid_pgns/2023_pgn/T!HV-SrsSzabi-Balu-2023-01-30-1835.pgn
valid_pgns/2023_pgn/HV-smburgan-spfish-2023-01-05-2115.pgn
valid_pgns/2023_pgn/T!HV-SOLFAREMI-smatt-2023-05-09-1002.pgn
valid_pgns/2023_pgn/T!HV-mangouste-RoXar-2023-03-06-2033.pgn
valid_pgns/2023_pgn/HV-mangouste-Samax-2023-03-01-2033.pgn
valid_pgns/2023_pgn/HV-superqwert-Gotcha-2023-01-06-0825.pgn
valid_pgns/2023_pgn/T!HV-spfish-hawk81-2023-05-01-1800.pgn
valid_pgns/2023_pgn/U!HV-MikhailTal-OrdepCubik-2023-02-04-1515.pgn
valid_pgns/2023_pgn/T!HV-xtof100-HeroMate-2023-04-16-1700.pgn
valid_pgns/2023_pgn/T!HV-SrsSzabi-ratcatcher-2023-05-07-1415.pgn
valid_pgns/2023_pgn/HV-OrdepCubik-Kaur50-2023-01-18-2009.pgn
valid_pgns/2023_pgn/T!HV-ringersoll-spfish-2023-05-04-1112.pgn
valid_pgns/2023_pgn/T!HV-hawk81-apolloq-2023-04-10-1806.pgn
valid_pgns/2023

valid_pgns/2021_pgn/HV-rachk-Echo45a-2021-11-29-1533.pgn
valid_pgns/2021_pgn/HV-Square623-RoXar-2021-05-25-1813.pgn
valid_pgns/2021_pgn/T!HV-InDusDus-CoalLover-2021-05-28-0925.pgn
valid_pgns/2021_pgn/HV-RoXar-Drikke-2021-04-12-1915.pgn
valid_pgns/2021_pgn/HV-Bigdog21-gusstew-2021-02-07-2251.pgn
valid_pgns/2021_pgn/HV-Hiligatio-Lucertious-2021-06-23-0117.pgn
valid_pgns/2021_pgn/T!HV-SOLFAREMI-hawk81-2021-02-20-1423.pgn
valid_pgns/2021_pgn/T!HV-hawk81-ClaudiuMe-2021-02-20-1604.pgn
valid_pgns/2021_pgn/HV-xtof100-ArhaeG-2021-02-20-1521.pgn
valid_pgns/2021_pgn/U!HV-Oakley-Natereater-2021-08-05-0224.pgn
valid_pgns/2021_pgn/T!HV-tzimarou-gandac-2021-09-27-1703.pgn
valid_pgns/2021_pgn/T!HV-Myrmigi-Balu-2021-12-12-1602.pgn
valid_pgns/2021_pgn/T!HV-SOLFAREMI-D0D0-2021-06-03-1857.pgn
valid_pgns/2021_pgn/HV-Bigdog21-gusstew-2021-02-06-1050.pgn
valid_pgns/2021_pgn/T!HV-lambda22-neighlles-2021-07-15-1604.pgn
valid_pgns/2021_pgn/HV-sjarel-ringersoll-2021-02-20-1237.pgn
valid_pgns/2021_pgn/T!HV-hawk81

valid_pgns/2022_pgn/T!HV-MikhailTal-SrsSzabi-2022-04-19-1553.pgn
valid_pgns/2022_pgn/U!HV-MikhailTal-Eucalyx-2022-03-27-0127.pgn
valid_pgns/2022_pgn/T!HV-Happykiwi-CrazyBee-2022-03-30-1307.pgn
valid_pgns/2022_pgn/HV-iLuvBugz-Connor1120-2022-09-16-0215.pgn
valid_pgns/2022_pgn/T!HV-InDusDus-SrsSzabi-2022-07-20-1457.pgn
valid_pgns/2022_pgn/HV-Balu-Laci55555-2022-04-06-0753.pgn
valid_pgns/2022_pgn/T!HV-neighlles-HappyKiwi-2022-07-08-1414.pgn
valid_pgns/2022_pgn/HV-Kaur50-SrsSzabi-2022-02-27-1942.pgn
valid_pgns/2022_pgn/T!HV-Cockroach-renoceros-2022-05-14-1437.pgn
valid_pgns/2022_pgn/T!HV-Loizz-D0D0-2022-02-23-1529.pgn
valid_pgns/2022_pgn/T!HV-Jewdoka-Gandac-2022-10-08-1336.pgn
valid_pgns/2022_pgn/HV-GadgetIV-Vinny2puds-2022-09-12-1809.pgn
valid_pgns/2022_pgn/HV-floipy-kethury-2022-06-25-2122.pgn
valid_pgns/2022_pgn/T!HV-Gandac-csigeee-2022-02-26-0959.pgn
valid_pgns/2022_pgn/T!HV-TedTheBear-MaxShark-2022-06-05-2028.pgn
valid_pgns/2022_pgn/T!HV-Jewdoka-spfish-2022-08-19-1759.pgn
valid_pgns/2

valid_pgns/2022_pgn/HV-Kaur50-HappyKiwi-2022-12-22-1929.pgn
valid_pgns/2022_pgn/HV-Kaur50-Loizz-2022-04-16-1214.pgn
valid_pgns/2022_pgn/T!HV-zsolti866-Kaur50-2022-06-11-1931.pgn
valid_pgns/2022_pgn/HV-RonDHuman-aradarbel-2022-10-21-1944.pgn
valid_pgns/2022_pgn/T!HV-susmumus-JonasK-2022-02-13-1829.pgn
valid_pgns/2022_pgn/HV-iLuvBugz-Martha-2022-09-16-0404.pgn
valid_pgns/2022_pgn/HV-Zyevrah-flammie-2022-03-31-0713.pgn
valid_pgns/2022_pgn/T!HV-SrsSzabi-stuart500-2022-03-11-1906.pgn
valid_pgns/2022_pgn/HV-kpeev3-MikhailTal-2022-04-13-2253.pgn
valid_pgns/2022_pgn/HV-MoeBro-Poppa-2022-03-13-1942.pgn
valid_pgns/2022_pgn/T!HV-hawk81-Kaur50-2022-05-01-1903.pgn
valid_pgns/2022_pgn/HV-superqwert-stepanzo-2022-10-14-0626.pgn
valid_pgns/2022_pgn/HV-Kaur50-HappyKiwi-2022-10-29-1756.pgn
valid_pgns/2022_pgn/HV-Kaur50-HappyKiwi-2022-12-09-2144.pgn
valid_pgns/2022_pgn/T!HV-hawk81-Eucalyx-2022-08-14-0708.pgn
valid_pgns/2022_pgn/T!HV-Marko60-mangouste-2022-06-17-1917.pgn
valid_pgns/2022_pgn/T!HV-zsolti866

In [7]:
len(all_games)

3858

In [8]:
def states_as_3d_array(states):
    output = []
    offset = (13,13)
    for state in states:
        grid = np.zeros((28,28), dtype='int32')
        for coords, stack in state:
            grid[coords[0]+offset[0]][coords[1]+offset[1]] = get_tile_encoding(stack[-1])
        output.append(grid)
    return output

In [9]:
states, final_state = moves_as_board_states(all_games[0])

In [10]:
x = states_as_3d_array(states)

In [11]:
print(len(states), len(x))

36 36


In [12]:
all_games_as_arrays = []
for i in all_games:
    states, _ = moves_as_board_states(i)
    all_games_as_arrays.append(states_as_3d_array(states))

In [13]:
len(all_games_as_arrays)

3858

In [14]:
game_1 = all_games_as_arrays[0]
game_tensor = torch.tensor(game_1)

/tmp/ipykernel_336629/3136187248.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  game_tensor = torch.tensor(game_1)


In [15]:
game_tensor.shape

torch.Size([36, 28, 28])

In [ ]:
class FirstNet(nn.Module):
    